# 语料数据分析

In [1]:
import os
import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
NEWS_DATASET_PATH = os.path.join(BASE_DIR, 'text_summ/datasets/sqlResult_1558435.csv')

In [4]:
BASE_DIR

'/Users/hotbaby/code/github/text-summarization'

## 新闻语料数据

In [5]:
news_dataset = pd.read_csv(NEWS_DATASET_PATH)

In [6]:
news_dataset.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


News datasets features

In [7]:
import json

In [8]:
features = news_dataset['feature']

feature_json = []
for feature in features:
    feature_json.append(json.loads(feature))

news_features = pd.DataFrame(feature_json)

In [9]:
news_features.head(1)

,all_type,clickNum,commentNum,favoriteNum,forwardNum,id,imgUrls,is_hot,issuing_department,joinNum,...,related_areas,reported_area,rich_label,shareNum,site,site_sign,summary,type,url,word_count
0,NaN,19920,37,NaN,NaN,NaN,http://static.cnbetacdn.com/thumb/mini/article...,2,NaN,20007,...,NaN,NaN,NaN,0,cnbeta,cnbeta,MIUI 9或许真的要来了！日前，微博数码博主@罗逸文V曝光了MIUI9首批支持机型，共有1...,科技,http://www.cnbeta.com/articles/tech/623597.htm,NaN


In [10]:
print(news_dataset['content'][1])

骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。
报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。
资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。
按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。
当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。



In [11]:
print(news_features['summary'][1])

尽管Intel最近以“涉嫌侵权”不点名批评微软和高通合作在骁龙835/820平台上实现对x86的模拟，但微软已经表示不会受影响，将为了广大消费者把此事做好。据外媒报道，微软最新的动态揭示，他们正在精益求精。为了平衡功耗，big.little架构在移动处理器上非常常见，这对习惯了Intel/AMD x86的Windows来说，是个新鲜事物。


In [12]:
news_dataset.iloc[-1]

id                                                         1
author                                                王子江 张寒
source                                                   新华社
content    　　新华社德国杜塞尔多夫６月６日电题：乒乓女球迷　\n　　新华社记者王子江、张寒\n　　熊老...
feature    {"type":"体育","site":"新华社","url":"http://home.x...
title                                           （体育）题：乒乓女球迷　
url        http://home.xinhua-news.com/gdsdetailxhsnew/22...
Name: 89610, dtype: object

In [13]:
news_dataset['title'].iloc[-1]

'（体育）题：乒乓女球迷\u3000'

In [14]:
content = news_dataset['content'].iloc[-1]
print('\n'.join(content.split('\\n')))

　　新华社德国杜塞尔多夫６月６日电题：乒乓女球迷　
　　新华社记者王子江、张寒
　　熊老师离开上海前，特意花一千多元买了一只张继科代言的球拍，准备在世界锦标赛期间他赢了比赛后，能在球拍上签个名。但没想到张继科在３２强赛中，输给了韩国的李尚洙。
　　“拍子算是白拿过来了。”熊老师摇头，“就算是信仰充值了。”
　　“信仰充值”是乒乓球粉丝圈非常流行的词汇，简单来说，就是为自己喜欢的明星或球队花钱。
　　“熊老师”是这位２２岁的成都姑娘的“艺名”，“艺名”这一称呼也是她们自己形容的。熊老师的真实身份是上海一所名牌大学会计专业的大四学生，其他同学正在抓紧毕业找工作，她一个人溜到德国观看世乒赛。尽管带着张继科的球拍，但熊老师并不是张的铁粉，她并不喜欢张继科输了比赛之后，一堆“科迷”在赛场外抱头痛哭的场景，而显得比较理智。但是因为“科迷”数量众多，让她这种散户，也有些说不出的羡慕。
　　“他们竟然连续两天买断了一面看台上所有的门票，为的是挂起带有张继科名字的旗子，两天就要花４、５万人民币。”熊老师直喊过分。
　　熊老师与其他１０名中国队女球迷挤在一套三居室里，厨房的柜子里塞了很多方便面。至少三个人挤在一个房间里，条件的确差了点，但住宿费平摊下来每天只有大约２００元人民币，这样让大家心里平衡了很多。
　　房间里的所有人都是在网上认识的，大家也并不全知道对方真实的姓名和身份。“我们都用艺名。”来自广州的茶茶说。茶茶已经工作了几年，这次也是利用假期出来的。
　　所有人都是提前大约半年从国际乒联的官网上买了比赛的通票，可以观看全程所有的比赛，票价大约两千块人民币。５月底６月初并不是旅游旺季，机票也不是太贵，这让她们都很庆幸。
　　里面最幸运的应该是井井，她这次来的身份是一家时尚媒体的记者，她申请到了记者证，每天背着一个大相机在场地里边拍照边看球。她提前大约一个星期就来到了德国，比赛开始前她去附近几个城市玩了玩，这一趟世乒赛之旅非常充实。
　　十几个人中最不像球迷的是一位来自香港的女大学生，看上去文静内向，其他人都叫她香港妹。香港妹与熊老师一样，都是学的会计专业。她告诉父母这次出来是到德国看朋友，住宿和球票的花费都是平时做家教挣的，父母支援了机票钱，从香港到杜塞尔多夫往返只有三千多元人民币，让其他内地球迷非常羡慕。
　　香港妹以前并不懂乒乓球，去年里约奥运会时，一家人边吃饭边看电视，正好

In [15]:
print(news_dataset['title'].iloc[-1])

（体育）题：乒乓女球迷　


In [16]:
print(news_features['summary'].iloc[-1])

　　新华社德国杜塞尔多夫６月６日电 题：乒乓女球迷　 　　新华社记者王子江、张寒 　　熊老师离开上海前，特意花一千多元买了一只张继科代言的球拍，准备在世界锦标赛期间他赢了比赛后，能在球拍上签个名。但没想到张继科在３２强赛中，输给了韩国的李尚洙。 　　“拍子算是


### 清洗数据

In [17]:
news_dataset.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [18]:
title_conent_df = news_dataset.loc[:, ['title', 'content']]

In [123]:
import re

def strip_str(string):
    strip_symbols = ['，', '。', '《', '》', ' ', '!', '！' '、', '.', '；', '~', 
                     '、', '”', '“', ';', '%', '\n', '\\n', '（', '）', '/', '——',
                    '：']
    # TODO 处理 \u3000
    for symbol in strip_symbols:
        string = string.replace(symbol, '')

    return re.sub(re.compile(r'\\u\d{4}'), '', '%r' % string)[2:-2]

In [66]:
strip_str(news_dataset['content'].iloc[-1])

'\u3000\u3000新华社德国杜塞尔多夫６月６日电题乒乓女球迷\u3000\u3000\u3000新华社记者王子江张寒\u3000\u3000熊老师离开上海前特意花一千多元买了一只张继科代言的球拍准备在世界锦标赛期间他赢了比赛后能在球拍上签个名但没想到张继科在３２强赛中输给了韩国的李尚洙\u3000\u3000拍子算是白拿过来了熊老师摇头就算是信仰充值了\u3000\u3000信仰充值是乒乓球粉丝圈非常流行的词汇简单来说就是为自己喜欢的明星或球队花钱\u3000\u3000熊老师是这位２２岁的成都姑娘的艺名艺名这一称呼也是她们自己形容的熊老师的真实身份是上海一所名牌大学会计专业的大四学生其他同学正在抓紧毕业找工作她一个人溜到德国观看世乒赛尽管带着张继科的球拍但熊老师并不是张的铁粉她并不喜欢张继科输了比赛之后一堆科迷在赛场外抱头痛哭的场景而显得比较理智但是因为科迷数量众多让她这种散户也有些说不出的羡慕\u3000\u3000他们竟然连续两天买断了一面看台上所有的门票为的是挂起带有张继科名字的旗子两天就要花４５万人民币熊老师直喊过分\u3000\u3000熊老师与其他１０名中国队女球迷挤在一套三居室里厨房的柜子里塞了很多方便面至少三个人挤在一个房间里条件的确差了点但住宿费平摊下来每天只有大约２００元人民币这样让大家心里平衡了很多\u3000\u3000房间里的所有人都是在网上认识的大家也并不全知道对方真实的姓名和身份我们都用艺名来自广州的茶茶说茶茶已经工作了几年这次也是利用假期出来的\u3000\u3000所有人都是提前大约半年从国际乒联的官网上买了比赛的通票可以观看全程所有的比赛票价大约两千块人民币５月底６月初并不是旅游旺季机票也不是太贵这让她们都很庆幸\u3000\u3000里面最幸运的应该是井井她这次来的身份是一家时尚媒体的记者她申请到了记者证每天背着一个大相机在场地里边拍照边看球她提前大约一个星期就来到了德国比赛开始前她去附近几个城市玩了玩这一趟世乒赛之旅非常充实\u3000\u3000十几个人中最不像球迷的是一位来自香港的女大学生看上去文静内向其他人都叫她香港妹香港妹与熊老师一样都是学的会计专业她告诉父母这次出来是到德国看朋友住宿和球票的花费都是平时做家教挣的父母支援了机票钱从香港到杜塞尔多夫往返只有三千多元人民币让其他内地球迷非常羡慕\u3000\u3

处理清洗数据

In [21]:
def drop_na_rows(dataset, columns=('title', 'content')):
    """
    drop na rows.

    :param dataset, pandas DataFrame
    :return drop na dataset, pandas DataFrame
    """

    for col in columns:
        dataset = dataset[dataset[col].notnull()]

    return dataset

输出到文件

In [124]:
NEWS_CORPUS_OUTPUT_PATH = os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.txt')

In [125]:
def preprocess(dataset, columns=['title', 'content']):
    # drop na rows
    df = drop_na_rows(news_dataset)

    # extract title and content data
    df = df.loc[:, columns]

    with open(NEWS_CORPUS_OUTPUT_PATH, 'w') as f:
        for title, content in df.values:
            # strip string
            title = strip_str(title)
            content = strip_str(content)

            f.write(title)
            f.write('\n')
            f.write(content)
            f.write('\n')

In [126]:
preprocess(news_dataset)

In [127]:
!ls ../text_summ/datasets/news_corpus.txt

../text_summ/datasets/news_corpus.txt


分词

In [26]:
import jieba

In [27]:
def cut(s):
    return list(jieba.cut(s))

In [59]:
cut(strip_str(news_dataset['content'][0]))[:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']